# Understanding Different Modes of Block Cipher 

Consider two messages, each of which owns n blocks: $M_1 = m_1||m_2||...||m_n$ and $M_2 = m'_1||m_2||...||m_n$, where $M_1$ and $M_2$ differ only in the first block. For instance, 

$M_1$ = "11335577 is my student ID. We will encrypt the message with different ways."  
$M_2$ = "22446688 is my student ID. We will encrypt the message with different ways."

Which of the following statements hold? Assume the same key is used for all encryption.

1) If $M_1$ and $M_2$ are encrypted using `Electronic Code Book (ECB)`, none of the cipher text blocks will repeat between each message.
2) If $M_1$ and $M_2$ are encrypted using `Cipher Block Chaining (CBC)`, but the same Initialization Vector (IV) is used for encrypting both messages, none of the cipher text blocks will repeat between each message.

In [ ]:
M1 = b"11335577 is my student ID. We will encrypt the message in different ways."
M2 = b"22446688 is my student ID. We will encrypt the message in different ways."

In [ ]:
!python -m pip install cryptography

## Available modes in OpenSSL

aes-[128|192|256]-cbc  128/192/256 bit AES in CBC mode  
aes[128|192|256]       Alias for aes-[128|192|256]-cbc  
aes-[128|192|256]-cfb  128/192/256 bit AES in 128 bit CFB mode  
aes-[128|192|256]-cfb1 128/192/256 bit AES in 1 bit CFB mode  
aes-[128|192|256]-cfb8 128/192/256 bit AES in 8 bit CFB mode  
aes-[128|192|256]-ctr  128/192/256 bit AES in CTR mode  
aes-[128|192|256]-ecb  128/192/256 bit AES in ECB mode  
aes-[128|192|256]-ofb  128/192/256 bit AES in OFB mode

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7

In [ ]:
import random

SEED = 0
BLOCK_SIZE = 128

# seed
random.seed(0)

# generate random key and iv
KEY = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
IV = bytes(random.randrange(256) for _ in range(BLOCK_SIZE // 8))
print(KEY.hex(), IV.hex())

In [ ]:
# create padder that required by ECB and CBC modes, https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/
padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M1) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

padder, unpadder = PKCS7(BLOCK_SIZE).padder(), PKCS7(BLOCK_SIZE).unpadder()
padded_data = padder.update(M2) + padder.finalize()
print(padded_data)
print(unpadder.update(padded_data) + unpadder.finalize())

In [ ]:
def encrypt_then_decrypt(plaintext, mode=modes.ECB()):
    cipher = Cipher(algorithms.AES(KEY), mode)
    encryptor, decryptor = cipher.encryptor(), cipher.decryptor()
    padder, unpadder = PKCS7(128).padder(), PKCS7(128).unpadder()

    # encrypt
    if len(plaintext) % 16 != 0:
        padded_data = padder.update(plaintext) + padder.finalize()
    else:
        padded_data = plaintext

    ciphered_data = encryptor.update(padded_data) + encryptor.finalize()

    # decrypt
    decipered_data = decryptor.update(ciphered_data) + decryptor.finalize()
    if len(plaintext) % 16 != 0:
        unpadded_data = unpadder.update(decipered_data) + unpadder.finalize()
        assert plaintext == unpadded_data

    return ciphered_data

In [ ]:
import textwrap

color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=True, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

def visual_hex_diff(bstr_1, bstr_2, hex_names=("HEX 1", "HEX 2")):
    SEP = "   |   "
    print("  ", f"{hex_names[0]}".ljust(16 + 7), hex_names[1], sep=SEP)
    # block level
    hex_1, hex_2 = textwrap.wrap(bstr_1.hex(), 16), textwrap.wrap(bstr_2.hex(), 16)
    for i, (block_1, block_2) in enumerate(zip(hex_1, hex_2)):
        # byte level
        block_1, block_2 = textwrap.wrap(block_1, 2), textwrap.wrap(block_2, 2)
        block_2 = [colorize(v2, "red" if v1 != v2 else "green") for v1, v2 in zip(block_1, block_2)]
        print(str(i).rjust(2), " ".join(block_1).ljust(16 + 7), " ".join(block_2).ljust( 16 + 7), sep=SEP)

In [ ]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.ECB())
ciphered_M2 = encrypt_then_decrypt(M2, modes.ECB())
visual_hex_diff(ciphered_M1, ciphered_M2, ["deciphered M1", "deciphered M2"])

In [ ]:
ciphered_M1 = encrypt_then_decrypt(M1, modes.CBC(IV))
ciphered_M2 = encrypt_then_decrypt(M2, modes.CBC(IV))
visual_hex_diff(ciphered_M1, ciphered_M2, ["deciphered M1", "deciphered M2"])